# Bootcamp DIO - Santander 2023 (ETL com Python)

**Objetivo:** 

Esse projeto visa colocar em prática o conteúdo aprendido no bootcamp da DIO Santader 2023 sobre ETL com Python.
Para isso vou utilizar a API de previsão do tempo do INPE, para obter os dados de previsão do tempo para os próximos 4 dias de cada cidade que se encontra no arquivo csv.


**Passos a serem executados:**

1. Ler a planilha em formato CSV ('codCidades.csv'), com uma lista de cod das cidades :
  ```
cidade,cod
São Paulo,244
Salvador, 255
  ```
2. Consumir o endpoint de previsão do tempo do INPE`GET http://servicos.cptec.inpe.br/XML/cidade/{codCidade}/previsao.xml` para obter os dados de previsão do tempo para os próximos 4 dias de cada cidade.
3. Depois vou tratar as informações obtidas da API do INPE.
4. Gravar as informações em um arquivo csv no formato abaixo:
  ```
Cidade,Data,Tempo,Máxima,Mínima
São Paulo,25/08/2023,pn,21,16,8.0
São Paulo,26/08/2023,c,17,13,8.0
São Paulo,27/08/2023,c,16,12,8.0
São Paulo,28/08/2023,pn,16,12,8.0
  ```

## Importa as bibliotecas

Importa as bibliotecas a serem usadas no projeto

In [11]:
import pandas as pd
import requests
import csv
import xml.etree.ElementTree as ET
from datetime import datetime

## **E**xtract

Extraia a lista com os códigos das cidades a serem pesquisadas

In [12]:
df = pd.read_csv('codCidades.csv')
cod_cidades = df['cod'].tolist()
print(cod_cidades)

[244, 242, 241]


In [13]:
# Url da api do INPE
inpe_api_url = 'http://servicos.cptec.inpe.br/XML'

def obter_previsao(cod_cidade):
    response = requests.get(f'{inpe_api_url}/cidade/{cod_cidade}/previsao.xml')    
    return response.content if response.status_code == 200 else None


xmls_content = [data for cod in cod_cidades if (data := obter_previsao(cod)) is not None]
print(xmls_content)

[b"<?xml version='1.0' encoding='ISO-8859-1'?><cidade><nome>S\xe3o Paulo</nome><uf>SP</uf><atualizacao>2023-08-24</atualizacao><previsao><dia>2023-08-25</dia><tempo>pn</tempo><maxima>21</maxima><minima>16</minima><iuv>8.0</iuv></previsao><previsao><dia>2023-08-26</dia><tempo>c</tempo><maxima>17</maxima><minima>13</minima><iuv>8.0</iuv></previsao><previsao><dia>2023-08-27</dia><tempo>c</tempo><maxima>16</maxima><minima>12</minima><iuv>8.0</iuv></previsao><previsao><dia>2023-08-28</dia><tempo>pn</tempo><maxima>16</maxima><minima>12</minima><iuv>8.0</iuv></previsao></cidade>", b"<?xml version='1.0' encoding='ISO-8859-1'?><cidade><nome>Salvador</nome><uf>BA</uf><atualizacao>2023-08-24</atualizacao><previsao><dia>2023-08-25</dia><tempo>pn</tempo><maxima>31</maxima><minima>23</minima><iuv>9.0</iuv></previsao><previsao><dia>2023-08-26</dia><tempo>pn</tempo><maxima>32</maxima><minima>22</minima><iuv>9.0</iuv></previsao><previsao><dia>2023-08-27</dia><tempo>pn</tempo><maxima>31</maxima><minima>

## **T**ransform

Trata as informações obtidas da Api do INPE.

In [14]:
def formata_data(data_original):
    data_obj = datetime.strptime(data_original, "%Y-%m-%d")
    return data_obj.strftime("%d/%m/%Y")

def trata_dados_xml(xml_content):
    
    previsoes = []

    root = ET.fromstring(xml_content)

    # Pega o nome da cidade
    nome_cidade = root.find("nome").text
    
    #Obtém as previsões do tempo da cidade
    for previsao_element in root.findall("previsao"):
   
        data = formata_data(previsao_element.find("dia").text)
        tempo = previsao_element.find("tempo").text
        maxima = previsao_element.find("maxima").text
        minima = previsao_element.find("minima").text
        iuv = previsao_element.find("iuv").text
    
        previsao = [nome_cidade, data, tempo, maxima, minima, iuv]   
        previsoes.append(previsao)

    return previsoes


previsoes_dict = [data for xml in xmls_content if (data := trata_dados_xml(xml)) is not None]
print(previsoes_dict)

[[['São Paulo', '25/08/2023', 'pn', '21', '16', '8.0'], ['São Paulo', '26/08/2023', 'c', '17', '13', '8.0'], ['São Paulo', '27/08/2023', 'c', '16', '12', '8.0'], ['São Paulo', '28/08/2023', 'pn', '16', '12', '8.0']], [['Salvador', '25/08/2023', 'pn', '31', '23', '9.0'], ['Salvador', '26/08/2023', 'pn', '32', '22', '9.0'], ['Salvador', '27/08/2023', 'pn', '31', '22', '10.0'], ['Salvador', '28/08/2023', 'pn', '31', '22', '10.0']], [['Rio de Janeiro', '25/08/2023', 'ci', '25', '21', '7.0'], ['Rio de Janeiro', '26/08/2023', 'c', '22', '19', '8.0'], ['Rio de Janeiro', '27/08/2023', 'c', '20', '16', '8.0'], ['Rio de Janeiro', '28/08/2023', 'c', '20', '16', '8.0']]]


## **L**oad

Salva as informações das previsões do tempo das cidades em um arquivo csv .

In [15]:
with open("previsoes.csv", "w", newline="", encoding="utf-8") as csvfile:
    writer = csv.writer(csvfile)
    
    # Adicionar o cabeçalho ao CSV
    cabecalho = ["Cidade", "Data", "Tempo", "Máxima", "Mínima"]
    writer.writerow(cabecalho)
    
    # Adicionar os dados
    for previsao_list in previsoes_dict:
        for linha in previsao_list:
            writer.writerow(linha)

print("Arquivo CSV salvo com sucesso!")

Arquivo CSV salvo com sucesso!
